# Проект по SQL

### ЦЕЛЬ: проанализировать информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг

### ЗАДАЧИ:
    1. определить количество книг изданных после 1 января 2000 года;
    2. для каждой книги расчитать количество обзоров и среднюю оценку;
    3. определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
    4. определить автора с самой высокой средней оценкой книг — учитывайть только книги с 50 и более оценками;
    5. посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

#### Описание данных
Таблица books (содержит данные о книгах):
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors (содержит данные об авторах):
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers (содержит данные об издательствах):
- publisher_id — идентификатор издательства;
- publisher — название издательства

Таблица ratings (содержит данные о пользовательских оценках книг):
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews (содержит данные о пользовательских обзорах):
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

In [1]:
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных


In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 


In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

##### ТАБЛИЦА books

In [5]:
pd.io.sql.read_sql('''SELECT * FROM books LIMIT(1) ''', con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


##### ТАБЛИЦА authors

In [6]:
pd.io.sql.read_sql('''SELECT * FROM authors LIMIT(1) ''', con = engine) 

,author_id,author
0,1,A.S. Byatt


##### ТАБЛИЦА publishers

In [7]:
pd.io.sql.read_sql('''SELECT * FROM publishers LIMIT(1) ''', con = engine) 

,publisher_id,publisher
0,1,Ace


##### ТАБЛИЦА ratings

In [8]:
pd.io.sql.read_sql('''SELECT * FROM ratings LIMIT(1) ''', con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


##### ТАБЛИЦА reviews

In [9]:
pd.io.sql.read_sql('''SELECT * FROM reviews LIMIT(1) ''', con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


##### 1. сколько книг вышло после 1 января 2000 года

In [10]:
pd.io.sql.read_sql('''SELECT Count(Distinct book_id) AS cnt FROM books 
                      WHERE publication_date::date>='2000-01-01' ''', con = engine) 

,cnt
0,821


Всего выпущено книг:

In [11]:
pd.io.sql.read_sql('''SELECT Count(Distinct book_id) AS cnt FROM books ''', con = engine) 

,cnt
0,1000


Из 1000 книг после 1 января 2000 года было издано 821 книг

##### 2. Для каждой книги посчитайте количество обзоров и среднюю оценку

In [12]:
pd.io.sql.read_sql('''SELECT reviews.book_id, Count(DISTINCT reviews.review_id) AS reviews, 
                      AVG(ratings.rating) AS avr_rating
                      FROM books, reviews INNER JOIN  ratings ON reviews.book_id=ratings.book_id
                      GROUP BY reviews.book_id
                      ORDER BY avr_rating DESC''', con = engine)

,book_id,reviews,avr_rating
0,972,2,5.00
1,513,2,5.00
2,901,2,5.00
3,136,2,5.00
4,86,2,5.00
...,...,...,...
989,915,3,2.25
990,371,2,2.00
991,316,2,2.00
992,202,3,2.00


В таблице представлена по каждой книге информация о количестве обзоров и средний рейтинг

##### 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [13]:
pd.io.sql.read_sql('''SELECT publishers.publisher, COUNT(DISTINCT books.title) AS cnt_books FROM books
                      INNER JOIN  publishers ON publishers.publisher_id=books.publisher_id
                      WHERE num_pages>=50
                      GROUP BY publishers.publisher
                      ORDER BY cnt_books DESC
                      LIMIT (1)
                      ''', con = engine) 

,publisher,cnt_books
0,Penguin Books,42


Penguin Books -  издательство, которое выпустило наибольшее число книг толще 50 страниц

##### 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [14]:
pd.io.sql.read_sql('''SELECT authors.author, AVG(S.avg_rating) AS avg_rating,
                      COUNT(books.book_id) AS cnt_book FROM books
                      INNER JOIN  authors ON authors.author_id=books.author_id
                      INNER JOIN
                      (SELECT book_id, COUNT(rating) AS cnt_rating, SUM(rating) AS sum_rating, AVG(rating) AS avg_rating 
                      FROM ratings
                      WHERE book_id in (SELECT book_id FROM ratings
                      GROUP BY book_id
                      HAVING COUNT(rating)>=50)
                      GROUP BY book_id) AS S ON S.book_id=books.book_id
                      GROUP BY authors.author
                      ORDER BY avg_rating DESC
                     ''', con = engine) 

,author,avg_rating,cnt_book
0,J.K. Rowling/Mary GrandPré,4.283844,4
1,Markus Zusak/Cao Xuân Việt Khương,4.264151,1
2,J.R.R. Tolkien,4.258446,2
3,Louisa May Alcott,4.192308,1
4,Rick Riordan,4.080645,1
5,William Golding,3.901408,1
6,J.D. Salinger,3.825581,1
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474,1
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879,1
9,Dan Brown,3.754540,2


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг среди книг с 50 и более оценками

##### 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [15]:
pd.io.sql.read_sql(''' SELECT AVG(Q.sum_rev) FROM
                      (SELECT SUM(S.cnt_rew) AS sum_rev FROM
                      (SELECT books.book_id, ratings.username,COUNT(ratings.rating_id) AS cnt_rat,COUNT(reviews.review_id) AS cnt_rew FROM books
                      LEFT JOIN ratings ON ratings.book_id=books.book_id
                      LEFT JOIN reviews ON reviews.book_id=books.book_id AND ratings.username=reviews.username
                      GROUP BY books.book_id, ratings.username, reviews.username
                      ORDER BY cnt_rat DESC) AS S
                      GROUP BY S.username
                      HAVING SUM (S.cnt_rat)>50)AS Q
                      ''', con = engine)

,avg
0,24.333333


В среднем пользователи которые поставили более 50 оценок дали 24 обзора книг